In [33]:
import sys, os
sys.path.append('..')

In [34]:
import pandas as pd
import swifter
from text_processing import TextProcessing

In [35]:
# %load_ext autoreload
# %autoreload 2

from load_config import config
# config

### Loading Raw Catalog Data

In [36]:
catalog = pd.read_csv("/Users/mukul4.verma/Documents/workspace/catalog_indexer/data/jiomart/raw/catalog_cleaned.csv")
catalog.shape

(121775, 14)

In [37]:
catalog.columns

Index(['Unnamed: 0', 'productid', 'productskuid', 'New Product Name',
       'Quantity', 'New_level1', 'New_level2', 'Valued_level2', 'New_level3',
       'brandname', 'orig_score', 'aggr_score', 'processed_title',
       'processed_title_final'],
      dtype='object')

In [38]:
catalog.head()

,Unnamed: 0,productid,productskuid,New Product Name,Quantity,New_level1,New_level2,Valued_level2,New_level3,brandname,orig_score,aggr_score,processed_title,processed_title_final
0,0,100026392,10030996.0,24 mantra organic cloves,50 g,masala,spices & masalas,spices & masalas,whole spices,24 Mantra,0.0,0.0,24 mantra organic clove,mantra organic clove
1,1,100145952,10157693.0,3 ply face mask,10 pcs,NaN,masks,NaN,face masks,Netplay,0.0,0.0,3 ply face mask,ply face mask
2,2,100139524,10151265.0,command white l plastic utility hook 1 hook 2 ...,3M,home,home improvement,NaN,NaN,3M,0.0,0.0,command white l plastic utility hook 1 hook 2 ...,command white plastic utility hook hook strip
3,3,100000074,10000117.0,a and w diet root beer aged vanilla,355 ml,NaN,soft drinks,soft drinks,aerated drink,A & W,0.0,0.0,a and w diet root beer age vanilla,and diet root beer age vanilla
4,4,100088833,10101294.0,armr 100 herbal blackberry antihangover drink,60 ml,NaN,NaN,NaN,over the counter remedies,ARMR,0.0,0.0,armr 100 herbal blackberry antihangover drink,armr herbal blackberry antihangover drink


In [39]:
relevant_columns = ['processed_title_final', 'New_level1', 'New_level2', 'Valued_level2', 'New_level3', 'brandname']
column_eda_df = list()
for column in catalog.columns:
    column_eda = dict()
    if column in relevant_columns:
        column_eda['column'] = column
        column_eda['null_percentage'] = int(100*catalog[column].isnull().sum()/catalog.shape[0])
        column_eda['total null count'] = catalog[column].isnull().sum()
        column_eda['# unique values'] = catalog[column].nunique()
        column_eda_df.append(column_eda)

column_eda_df = pd.DataFrame(column_eda_df)
column_eda_df

,column,null_percentage,total null count,# unique values
0,New_level1,79,96452,79
1,New_level2,13,16381,452
2,Valued_level2,48,58801,120
3,New_level3,12,14690,1007
4,brandname,0,7,6844
5,processed_title_final,0,26,94754


### Catalog Indexer Class

In [40]:
from load_config import config

import pandas as pd
from text_processing import TextProcessing
from compute_missing_attributes import ComputeMissingAttributes
from compute_ampersand_category import ComputeAmpersandCategory

class CatalogIndexer():

    def __init__(self, config, domain, catalog):
        self.config = config
        self.domain = domain
        self.catalog = catalog
        self._validate_catalog()

    def _validate_catalog(self):
        if not type(self.catalog) == pd.core.frame.DataFrame:
            raise Exception('catalog should be a pandas dataframe')
        if not all([input_column in self.catalog.column for input_column in config[self.domain]['input_columns']]):
            raise Exception('input columns mentioned in the config are not present in the given catalog')

    def index_catalog(self):
        
        if self.config[self.domain]['run_text_processor']:
            self.text_processor = TextProcessing()
            self.run_text_processor()

        if self.config[self.domain]['run_ampersand_category']:
            self.compute_ampersand_category = ComputeAmpersandCategory()
            self.run_ampersand_category()

        if self.config[self.domain]['run_missing_attribute']:
            self.compute_missing_attribute = ComputeMissingAttributes()
            self.run_missing_attribute()

        if self.config[self.domain]['run_merge_attributes']:
            pass
            

    def run_text_processor(self, catalog):
        pass

    def run_ampersand_category(self, catalog):
        pass

    def run_missing_attribute(self, catalog):
        self.catalog = self.compute_missing_attribute.compute(self.catalog, self.config[self.domain])

In [41]:
type(pd.DataFrame()) == pd.core.frame.DataFrame

True

In [42]:
# get_training_data_for_missing_column(data, 'New Product Name', 'New_level1')

## Classification pipeline

In [79]:
import pandas as pd

# data transformation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

# binary relevance
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier

# classification models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

# performance metric
from sklearn.metrics import f1_score

# model pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer 

# text processing library
# from text_processing import TextProcessing

# class TextProcessor(BaseEstimator):

#     def __init__(self, text_preprocessing_model, text_column):
#         self.text_column = text_column
#         self.text_preprocessing_model = text_preprocessing_model

#     def fit(self, documents, y=None):
#         return self

#     def transform(self, x_dataset):
#         x_dataset['cleaned_text'] = x_dataset[self.text_column].apply(lambda x: self.text_preprocessing_model.clean_text(x))
#         return x_dataset
import sister

class SentenceEmbedding():

    def __init__(self, embedding_type = None):
        # if embedding_type == 'bert':
        #     self.sentence_embedding = sister.BertEmbedding(lang="en")
        # else:
        #     self.sentence_embedding = sister.MeanEmbedding(lang="en")
        if embedding_type == 'bert':
            self.sentence_embedding = sentence_embedding
        else:
            self.sentence_embedding = sentence_embedding_bert

    def get_sentence_embedding(self, sentence):
        if type(sentence) == str:
            return self.sentence_embedding(sentence)
        else:
            return [self.sentence_embedding(str(sent)) for sent in list(sentence)]

class TextVectorizer(BaseEstimator):

    def __init__(self, text_column, vectorizer_algorithm, embedding_type = None):
        self.text_column = text_column
        self.vectorizer_algorithm = vectorizer_algorithm
        self.vectorizer = None
        self.sentence_embedding = SentenceEmbedding(embedding_type) if vectorizer_algorithm == 'sentence_embeddings' else None

    def fit(self, x_dataset, y=None):
        if self.vectorizer_algorithm == 'count_vectorizer':
            pass
        elif self.vectorizer_algorithm == 'tfidf_vectorizer':
            self.vectorizer = TfidfVectorizer()
            self.vectorizer.fit(x_dataset)
        elif self.vectorizer_algorithm == 'sentence_embeddings':
            pass
        else:
            raise Exception(f'invalid vectorizer_algorithm: {vectorizer_algorithm}')
        return self

    def transform(self, x_dataset):
        if self.vectorizer_algorithm == 'count_vectorizer':
            pass
        elif self.vectorizer_algorithm == 'tfidf_vectorizer':
            x_dataset = self.vectorizer.transform(x_dataset)
        elif self.vectorizer_algorithm == 'sentence_embeddings':
            self.sentence_embedding.get_sentence_embedding(x_dataset)
        else:
            raise Exception(f'invalid vectorizer_algorithm: {vectorizer_algorithm}')
        return x_dataset

class TextClassification():

    def __init__(self, data, text_column, label_column, classification_type, model_algorithm, vectorizer_algorithm):
        
        # assigning params to instance variable
        self.data = data
        self.text_column = text_column
        self.label_column = label_column
        self.classification_type = classification_type
        self.model_algorithm = model_algorithm
        self.vectorizer_algorithm = vectorizer_algorithm
        
        # self.text_preprocessing_model = text_preprocessing_model

        # process data
        self.process_data()

        #building pipeline
        self._build_model_pipeline()

    def process_data(self,):
        
        self.data = self.data[[self.text_column, self.label_column]]
        
        # transforming target label
        if self.classification_type == 'multi-class':
            self.encoder = LabelEncoder()
            self.data[self.label_column] = self.encoder.fit_transform(self.data[self.label_column])
        else:
            self.encoder = MultiLabelBinarizer()
            self.data[self.label_column] = self.encoder.fit_transform(self.data[self.label_column])

        # splitting of training and test data
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(self.data[self.text_column], self.data[self.label_column], test_size=0.2, shuffle=True, random_state=1)

    # building end to end pipeline
    def _build_model_pipeline(self,):

        pipeline_steps = list()
        pipeline_steps.append(('text_vectorizer', TextVectorizer(self.text_column, self.vectorizer_algorithm)))
        # pipeline_steps.append(('column_transformation', self._load_column_transformer()))

        if self.model_algorithm == 'LR' or self.model_algorithm == None:
            pipeline_steps.append(('LogisticRegression', OneVsRestClassifier(LogisticRegression())))

        elif self.model_algorithm == 'NB':
            pipeline_steps.append(('MultinomialNB', OneVsRestClassifier(MultinomialNB())))

        elif self.model_algorithm == 'SVC':
            pipeline_steps.append(('LinearSVC', OneVsRestClassifier(LinearSVC())))

        elif self.model_algorithm == 'XGB':
            pipeline_steps.append(('XGBClassifier', OneVsRestClassifier(XGBClassifier(verbosity = 0))))

        self.model_pipeline = Pipeline(steps=pipeline_steps)

    # pipeline component for transforming any columns in input dataframe
    def _load_column_transformer(self,):
        return None

    # training pipeline
    def train_pipeline(self,):
        self.model_pipeline.fit(self.train_x, self.train_y)

    # evaluating pipeline
    def evaluate_pipeline(self,):
        y_pred = self.model_pipeline.predict(self.test_x)
        metric_score = f1_score(self.test_y, y_pred, average="micro")
        print(f'F1 score for {self.model_algorithm}: {metric_score}')

    # prediction using trained pipeline
    def predict(self, input_text):
        input_text = [input_text]
        prediction = self.model_pipeline.predict(input_text)
        return self.encoder.inverse_transform(prediction)


## from cleaned dataframe to training data for pipeline

In [80]:
domain_config = config['jiomart_autosuggest']
domain_config

{'run_spell_check': False,
 'run_ampersand_category': False,
 'run_missing_attribute': True,
 'run_merge_attributes': False,
 'columns_for_text_processing': [],
 'columns_for_ampersand_category': [],
 'columns_for_missing_attributes': ['New_level1',
  'New_level2',
  'New_level3',
  'Valued_level2'],
 'columns_for_input_text_data': ['processed_title_final']}

### training using TF-IDF vectorizer

In [84]:
import pickle

INPUT_TEXT_COLUMN = 'input_text'
catalog[INPUT_TEXT_COLUMN] = catalog.apply(lambda row: " ".join(str(row[column]) for column in domain_config['columns_for_input_text_data']), axis = 1)
classification_pipelines = dict()
model_path = '/Users/mukul4.verma/Documents/workspace/catalog_indexer/src/models/classification_pipelines.pkl'

for column in domain_config['columns_for_missing_attributes']:
    print(f"Column: {column}")
    train_df = catalog[[column, 'input_text']]
    train_df.dropna(inplace=True)
    train_df = train_df[~train_df[column].str.contains('dummy')]
    print(train_df[column].nunique())

    for algo in ['LR', 'NB', 'SVC']:
        print(f'model_algo: {algo}')
        classification = TextClassification(train_df,INPUT_TEXT_COLUMN, column, 'multi-class', algo, 'tfidf_vectorizer')
        classification.train_pipeline()
        classification.evaluate_pipeline()
        classification_pipelines[f'{column}_{algo}'] = classification
        with open(model_path, 'wb') as dbfile:
            pickle.dump(classification_pipelines, dbfile) 

        print()
    print()
    # break
    # print(train_df.head())
    # print(train_df.shape)
    # break

Column: New_level1
78
model_algo: LR
F1 score for LR: 0.8966545919454507

model_algo: NB
F1 score for NB: 0.8670360110803325

model_algo: SVC
F1 score for SVC: 0.9477945876837843


Column: New_level2
451
model_algo: LR
F1 score for LR: 0.8295745400106245

model_algo: NB
F1 score for NB: 0.7698362872458588

model_algo: SVC
F1 score for SVC: 0.8566668276428261


Column: New_level3
1006
model_algo: LR
F1 score for LR: 0.6931812782133523

model_algo: NB
F1 score for NB: 0.544024708956997

model_algo: SVC
F1 score for SVC: 0.7445949156569256


Column: Valued_level2
119
model_algo: LR
F1 score for LR: 0.8834983228340014

model_algo: NB
F1 score for NB: 0.8399738198478278

model_algo: SVC
F1 score for SVC: 0.9109056696392048




### training using sentence embeddings

In [81]:
import pickle

INPUT_TEXT_COLUMN = 'input_text'
catalog[INPUT_TEXT_COLUMN] = catalog.apply(lambda row: " ".join(str(row[column]) for column in domain_config['columns_for_input_text_data']), axis = 1)
classification_pipelines = dict()
model_path = '/Users/mukul4.verma/Documents/workspace/catalog_indexer/src/models/classification_pipelines_sent_embed_bert.pkl'

for column in domain_config['columns_for_missing_attributes']:
    print(f"Column: {column}")
    train_df = catalog[[column, 'input_text']]
    train_df.dropna(inplace=True)

    for algo in ['LR', 'NB', 'SVC']:
        print(f'model_algo: {algo}')
        classification = TextClassification(train_df,INPUT_TEXT_COLUMN, column, 'multi-class', algo, 'sentence_embeddings')
        classification.train_pipeline()
        classification.evaluate_pipeline()
        classification_pipelines[f'{column}_{algo}'] = classification
        with open(model_path, 'wb') as dbfile:
            pickle.dump(classification_pipelines, dbfile) 

        print()
    print()
    # break
    # print(train_df.head())
    # print(train_df.shape)
    # break

Column: New_level1
model_algo: LR


KeyboardInterrupt: 

In [15]:
!pip3 install  sister

  Using cached sister-0.2.2-py3-none-any.whl (5.3 kB)
  Using cached mecab_python3-0.996.5-cp38-cp38-macosx_10_9_x86_64.whl (13.9 MB)
  Using cached torch-1.5.1-cp38-none-macosx_10_9_x86_64.whl (80.6 MB)
Processing /Users/mukul4.verma/Library/Caches/pip/wheels/2c/67/ed/d84123843c937d7e7f5ba88a270d11036473144143355e2747/progressbar-2.5-py3-none-any.whl
  Using cached transformers-2.11.0-py3-none-any.whl (674 kB)
Processing /Users/mukul4.verma/Library/Caches/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4/future-0.18.2-py3-none-any.whl
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached sentencepiece-0.1.96-cp38-cp38-macosx_10_6_x86_64.whl (1.1 MB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached tokenizers-0.7.0-cp38-cp38-macosx_10_10_x86_64.whl (1.2 MB)
You should consider upgrading via the '/Users/mukul4.verma/.local/share/virtualenvs/co-occurrence-gucGsh7Y/bin/python -m pip install --upgrade pip' command.


In [2]:
import sister
 
sentence_embedding = sister.MeanEmbedding(lang="en")
sentence_embedding_bert = sister.BertEmbedding(lang="en")
 
def get_phrase_embedding(phrase):
    return sentence_embedding(phrase)

def get_phrase_embedding_bert(phrase):
    return sentence_embedding_bert(phrase)

Loading model...


In [12]:
import numpy as np

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def phrase_sim(phrase_a, phrase_b, embedding_type = None):
    if embedding_type == 'bert':
        return cosine(get_phrase_embedding_bert(phrase_a), get_phrase_embedding_bert(phrase_b))
    else:
        return cosine(get_phrase_embedding(phrase_a), get_phrase_embedding(phrase_b))

In [13]:
print(len(get_phrase_embedding("haldiram")))
print(len(get_phrase_embedding_bert("haldiram")))

300
768


In [23]:
phrase_sim("rice", "kohinoor basmati rice")

0.82543105

In [24]:
class SentenceEmbedding():

    def __init__(self, embedding_type = None):
        if embedding_type == 'bert':
            self.sentence_embedding = sister.BertEmbedding(lang="en")
        else:
            self.sentence_embedding = sister.MeanEmbedding(lang="en")

    def get_sentence_embedding(self, sentence):
        return self.sentence_embedding(sentence)

In [26]:
se = SentenceEmbedding('bert')

In [62]:
len(se.get_sentence_embedding(['rice', 'potato'])[0])

768

In [67]:
se.sentence_embedding(['rice', 'potato'])

array([[ 0.7340733 ,  2.075869  , -0.46255904, ...,  0.50079906,
         0.750432  , -0.11352374],
       [ 1.0305916 ,  1.8630819 ,  0.18010414, ...,  0.41653   ,
         0.59846777,  0.531764  ]], dtype=float32)

In [69]:
se.sentence_embedding(['rice', 'potato'])

AttributeError: 'list' object has no attribute 'translate'

## old rough work

### Loading relevant modules

In [85]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['relevant_categories'])
print(multilabel_binarizer.classes_)

['APPAREL' 'FOOD GROCERY' 'FRUITS' 'HOME' 'HOME & PERSONAL CARE'
 'LIFESTYLE' 'PROTEIN' 'STAPLES' 'VEGETABLES']


In [12]:
# partition = int(0.8*data.shape[0])

target_labels = multilabel_binarizer.transform(data['relevant_categories'])
# train_labels, test_labels = target_labels[:partition], target_labels[partition:]

train_query, test_query, train_labels, test_labels = train_test_split(data['cleaned_query'], target_labels, test_size=0.2, shuffle=True, random_state=1)

# train_query = data['cleaned_query'].iloc[:partition].to_list()
# test_query = data['cleaned_query'].iloc[partition:].to_list()

vectorised_train_X = vectorizer.fit_transform(train_query)
vectorised_test_X = vectorizer.transform(test_query)

In [13]:
print(train_query.shape, train_labels.shape)
print(test_query.shape, test_labels.shape)

(10125,) (10125, 9)
(2532,) (2532, 9)


### Logistic classifier

In [17]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutputMultiOutputClassifier

# Performance metric
from sklearn.metrics import f1_score

In [15]:
train_labels[0]

array([0, 1, 1, 0, 1, 0, 0, 1, 1])

In [20]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

clf.fit(vectorised_train_X, train_labels)

# make predictions for validation set
y_pred = clf.predict(vectorised_test_X)

f1_score(test_labels, y_pred, average="micro")

0.7038653366583543

In [21]:
clf

OneVsRestClassifier(estimator=LogisticRegression())

In [22]:

lr = LogisticRegression()
clf = MultiOutputClassifier(lr)

clf.fit(vectorised_train_X, train_labels)

# make predictions for validation set
y_pred = clf.predict(vectorised_test_X)

f1_score(test_labels, y_pred, average="micro")

0.7038653366583543

In [23]:
clf

MultiOutputClassifier(estimator=LogisticRegression())

### Naive Bayes

In [26]:
from sklearn.naive_bayes import MultinomialNB

nbClassifier = MultiOutputClassifier(MultinomialNB())
nbClassifier.fit(vectorised_train_X, train_labels)
nbPreds = nbClassifier.predict(vectorised_test_X)
f1_score(test_labels, nbPreds, average="micro")

0.7138183517681883

### SVC

In [31]:
from sklearn.svm import LinearSVC

svmClassifier = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svmClassifier.fit(vectorised_train_X, train_labels)

svmPreds = svmClassifier.predict(vectorised_test_X)
f1_score(test_labels, svmPreds, average="micro")

0.7253632570853114

### XG Boost

In [32]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb_multi = OneVsRestClassifier(xgb)

xgb_multi.fit(vectorised_train_X, train_labels)

# make predictions for validation set
y_pred = xgb_multi.predict(vectorised_test_X)

f1_score(test_labels, y_pred, average="micro")

[23:57:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:57:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:57:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:57:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

0.603988603988604

In [36]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr_multi = OneVsRestClassifier(rfr)

rfr_multi.fit(vectorised_train_X, train_labels)

# make predictions for validation set
y_pred = rfr_multi.predict(vectorised_test_X)

f1_score(test_labels, y_pred, average="micro")

0.661923242777059

In [155]:
vectorised_train_X.

(10125, 4650)

In [16]:
import numpy as np

## Text Classification Class

In [30]:
# import sys, os
# sys.path.append('../src')

import pandas as pd

# binary relevance
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier

# classification models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

# performance metric
from sklearn.metrics import f1_score

# model pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer 

# text processing library
from helpers.text_processing import TextProcessing

class TextProcessor(BaseEstimator):

    def __init__(self, text_preprocessing_model, text_column):
        self.text_column = text_column
        self.text_preprocessing_model = text_preprocessing_model

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        x_dataset['cleaned_text'] = x_dataset[self.text_column].apply(lambda x: self.text_preprocessing_model.clean_text(x))
        return x_dataset


class TextClassification():

    def __init__(self, input_data_filepath, text_column, label_column, model_algorithm, classification_type, text_preprocessing_model = None):
        
        # assigning 
        self.classification_type = classification_type
        self.input_data_filepath = input_data_filepath
        self.text_column = text_column
        self.label_column = label_column
        self.text_preprocessing_model = text_preprocessing_model

        # loading data
        self.load_data()

        #building pipeline
        self._build_model_pipeline()

    def _load_data(self,):
        file_extension = self.raw_data_filepath.split(".")[-1]
        try:
            if file_extension == ".csv":
                self.raw_data = pd.read_csv(self.raw_data_filepath)

                self.train_x = 
                self.train_y = 
                self.test_x = 
                self.test_y = 
            else:
                raise ValueError (f'{file_extension} file type is not supported')
        except ValueError as ve:
            print(f'error occured while loading data: {ve}')
        except Exception as e:
            print(f'error occured while loading data: {e}')

    # building end to end pipeline
    def _build_model_pipeline():

        pipeline_steps = list()
        pipeline_steps.append(('text_preprocessing', TextProcessor()))
        pipeline_steps.append(('text_vectorizer', TextVectorizer()))
        pipeline_steps.append(('column_transformation', self._load_column_transformer()))

        if self.model_algorithm == 'LR' or self.model_algorithm == None:
            pipeline_steps.append(('LogisticRegression', LogisticRegression()))

        elif self.model_algorithm == 'NB':
            pipeline_steps.append(('MultinomialNB', MultinomialNB()))

        elif self.model_algorithm == 'SVC':
            pipeline_steps.append(('LinearSVC', LinearSVC()))

        elif self.model_algorithm == 'XGB':
            pipeline_steps.append(('XGBClassifier', XGBClassifier()))

        self.model_pipeline = Pipeline(steps=pipeline_steps)

    # pipeline component for transforming any columns in input dataframe
    def _load_column_transformer(self,):
        return None

    # training pipeline
    def train_pipeline(self,):
        self.model_pipeline.fit(self.train_x, self.train_y)

    # evaluating pipeline
    def evaluate_pipeline(self,):
        y_pred = self.model_pipeline.predict(self.test_x)
        metric_score = f1_score(self.test_y, y_pred, average="micro")
        print(f'F1 score for {self.model_algorithm}: {metric_score}')

    # prediction using trained pipeline
    def predict(self, input_text):
        input_text = [input_text]
        return self.model_pipeline.predict(input_text)


In [18]:
from sklearn.datasets import make_classification


In [19]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)


In [20]:
len(y)

1000

In [21]:
X

array([[ 1.89149379, -0.39847585,  1.63856893, ...,  0.58810926,
        -0.02542177, -0.52835426],
       [ 1.86913773, -0.56336215,  2.03411807, ..., -0.55633   ,
        -0.27340013,  0.72129251],
       [ 1.95259701, -2.83144572,  0.19055535, ..., -0.60957741,
        -2.07750191, -1.75469982],
       ...,
       [ 1.98033054,  1.37228804,  1.67376262, ...,  2.75687956,
        -0.20719842,  0.39299534],
       [-1.54305631, -0.27530218,  0.38744703, ...,  0.72769107,
         0.51983329, -1.92306657],
       [ 1.17970389, -3.3812155 , -0.1498426 , ...,  1.24640268,
        -4.00356845, -3.01027048]])